In [288]:
from langchain_community.chat_models.tongyi import ChatTongyi
from typing import TypedDict,Annotated
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph,START,END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ToolMessage

from langchain.tools import tool

llm = ChatTongyi(model="qwen-max",api_key="sk-9a1c78d0414e49aeb778f97bf608fef0")

In [289]:
@tool()
def search_flights(destination: str, date: str) -> dict:
    """查询指定日期飞往某地的航班信息。返回一个包含航班号和价格的字典，如果无航班则返回None。"""
    print(f"正在查询 {date} 前往 {destination} 的航班...")
    if destination == "新加坡":
        return {"flight_number": "SQ801", "price": 3000, "departure_time": "08:00"}
    return None
@tool()
def search_hotels(destination: str, check_in_date: str, check_out_date: str) -> list[dict]:
    """根据地点和日期查询酒店。返回酒店列表，如果无酒店则返回空列表。"""
    print(f"正在查询 {destination} 从 {check_in_date} 到 {check_out_date} 的酒店...")
    if destination == "新加坡" and check_in_date == "2025-10-27":
        return [{"name": "滨海湾金沙酒店", "price_per_night": 2500}, {"name": "莱佛士酒店", "price_per_night": 3500}]
    return []

@tool()
def book_flight_and_hotel(flight_number: str, hotel_name: str, guest_name: str) -> dict:
    """预订机票和酒店，这是一个事务性操作。"""
    print(f"正在为 {guest_name} 预订航班 {flight_number} 和酒店 {hotel_name}...")
    return {"status": "success", "booking_id": "12345ABC"}

In [290]:
class State(TypedDict):
    messages: Annotated[list,add_messages]

tools = [search_flights, search_hotels, book_flight_and_hotel]
tool_map = {"search_flights":search_flights, "search_hotels":search_hotels,"book_flight_and_hotel":book_flight_and_hotel}
tool_names = ["search_flights", "search_hotels", "book_flight_and_hotel"]
llm_with_tools = llm.bind_tools(tools)

history = []

template = """
    你是一个旅行规划助手，能够将用户的模糊指令（例如：“帮我预订下周一去新加坡的机票和酒店，住两晚，我的名字是王伟”）拆解成一个有序的执行计划，并使用以下工具根据用户的旅行需求帮助用户进行操作：
    1、search_flights：查询指定日期飞往某地的航班信息。
    2、search_hotels：根据地点和日期查询酒店。返回酒店列表，如果无酒店则返回空列表。
    3、book_flight_and_hotel：预订机票和酒店。

    用户输入：{input}
    历史对话记录：{history}
    当完成用户的需求则返回相应的出结果，并结束对话。
    """

prompt = ChatPromptTemplate.from_template(template)
# 创建agent，用来调用工具和输出结果
def agent(state: State):
    question = state["messages"][0]
    last_message = state["messages"][-1]
    history.append(last_message)
    chain = prompt|llm_with_tools
    response = chain.invoke({"input": question, "history": history})
    print(response.content)
    return {"messages": [response]}

In [291]:
def run_tool(state:State):
    """执行单个工具的调用"""
     # 从状态字典中获取最后一条消息
    last_message = state["messages"][-1]
    # print(last_message)
    tool_call = getattr(last_message, "tool_calls")
    # print(tool_call)
    tool_name = tool_call[0]["name"]
    # 从tool_map中获取对应的工具实例，若不存在则返回None
    tool = tool_map.get(tool_name)

    if not tool:
        raise ValueError(f"Tool {tool_name} not found")
    # 调用工具
    response = tool.invoke(tool_call[0]["args"])
    # 创建并返回ToolMessage对象，包含工具执行结果、调用ID和工具名称
    # print(response)
    if response == None:
        result = ToolMessage(content="未查找到相应的航班",tool_call_id=tool_call[0]["id"],name=tool_name)
    elif response == []:
        result = ToolMessage(content="未查找到相应的酒店",tool_call_id=tool_call[0]["id"],name=tool_name)
    else:
        result = ToolMessage(content=str(response),tool_call_id=tool_call[0]["id"],name=tool_name)
        # result = ToolMessage(content=str(response),tool_call_id=tool_call[0]["id"],name=tool_name)
    return {"messages": result}

In [292]:
from langgraph.prebuilt import tools_condition

# 构建图
graph_builder = StateGraph(state_schema=State)
graph_builder.add_node("agent",lambda state: agent(state))
graph_builder.add_node("call_tools",lambda state: run_tool(state))
graph_builder.add_edge(START,"agent")
graph_builder.add_conditional_edges(source="agent",path=tools_condition, path_map={"tools": "call_tools", END: END})
graph_builder.add_edge("call_tools","agent")
graph = graph_builder.compile()

In [294]:
# events = graph.stream({"messages": [{"role": "user", "content": "查询2025-10-25到新加坡的航班"}]})
events = graph.stream({"messages": [{"role": "user", "content": "帮王刚预定2025-10-25到新加坡的航班，以及2025-10-27日入住，2025-10-28退房的酒店"}]})


# 流式输出
for event in events:
    # 遍历事件中的值
    for value in event.values():
        # 检查是否有有效消息
        if "messages" not in value or not isinstance(value["messages"], list):
            continue

        # 获取最后一条消息
        last_message = value["messages"][-1]

        # 检查消息是否包含工具调用
        if hasattr(last_message, "tool_calls") and last_message.tool_calls:
            # 遍历工具调用
            for tool_call in last_message.tool_calls:
                # 检查工具调用是否为字典且包含名称
                if isinstance(tool_call, dict) and "name" in tool_call:
                    print(f"Calling tool: {tool_call['name']}")
            # 跳过本次循环
            continue

        # 检查消息是否有内容
        if hasattr(last_message, "content"):
            content = last_message.content

            # 情况1：工具输出（动态检查工具名称）
            if hasattr(last_message, "name") and last_message.name in tool_names:
                tool_name = last_message.name
                print(f"Tool Output [{tool_name}]: {content}")
            # 情况2：大模型输出（非工具消息）
            else:
                print(f"Assistant: {content}")
        else:
            print("Assistant: 未获取到相关回复")

为了帮助王刚预定2025-10-25到新加坡的航班，我将首先查询该日期飞往新加坡的航班信息。
Calling tool: search_flights
正在查询 2025-10-25 前往 新加坡 的航班...

Calling tool: search_hotels
正在查询 新加坡 从 2025-10-27 到 2025-10-28 的酒店...

Calling tool: book_flight_and_hotel
正在为 王刚 预订航班 SQ801 和酒店 滨海湾金沙酒店...
已经为王刚成功预订了2025年10月25日飞往新加坡的航班SQ801，以及在2025年10月27日至2025年10月28日期间入住滨海湾金沙酒店。预订ID为12345ABC。如果有其他需求或问题，请随时告知。
Assistant: 已经为王刚成功预订了2025年10月25日飞往新加坡的航班SQ801，以及在2025年10月27日至2025年10月28日期间入住滨海湾金沙酒店。预订ID为12345ABC。如果有其他需求或问题，请随时告知。
